In [ ]:
%matplotlib notebook


import numpy as np
import pandas as pd
import dask
import os


from operator import itemgetter

import numpy as np
import scipy
import matplotlib.pyplot as plt


import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split
from dask_ml.datasets import make_classification
from dask.diagnostics import ProgressBar
from dask_ml.model_selection import IncrementalSearchCV
import dask_ml.datasets
import dask_ml.cluster


import matplotlib.pyplot as plt
import numpy as np
from mlxtend.plotting import plot_decision_regions

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import joblib


In [ ]:
# Handle and preprocess data
X = feature_array
y = target_array.flatten()

# Split data into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scale data
sc = StandardScaler()
sc.fit(X_train)

# Transform (standardise) both X_train and X_test with mean and STD from
# training data
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)




## Simple logistic regression model

In [ ]:
parameters = {'penalty': ['l1', 'l2'], 'C': [0.5, 1, 2]}

lr = LogisticRegression()
tuned_lr = GridSearchCV(lr, parameters)

with ProgressBar():
    tuned_lr.fit(X_train, y_train)

In [ ]:
# Visualizing the score
pd.DataFrame(tuned_lr.cv_results_)

In [ ]:
# Get hyperparameters of best performing model
tuned_lr.best_params_

In [ ]:
# Fit the best classifier
clf = tuned_lr.best_params_
print(clf)
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))

## Create Scikit-Learn Estimator

In [ ]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]




In [ ]:
# Create a support vector machine (SVC) model using grid search
param_grid = {"C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)

In [ ]:
# Fit model using cluster or joblib
with joblib.parallel_backend('dask'):
    grid_search.fit(X, y)

In [ ]:
pd.DataFrame(grid_search.cv_results_).head()

In [ ]:
grid_search.predict(X)[:5]

In [ ]:
grid_search.score(X, y)

## Training on large data sets

In [ ]:
# generate some random dask arrays
X, y = dask_ml.datasets.make_blobs(n_samples=10000000,
                                   chunks=1000000,
                                   random_state=0,
                                   centers=3)
X = X.persist()
X

In [ ]:
# Create a support vector machine model (SVC) using grid search optimization
param_grid = {"C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)

In [ ]:
# Logistic regression model

lr = LogisticRegression()

lr.fit(data, labels)





In [ ]:
# Support Vector Machine model

# Train linear SVC
svm_linear = SVC(kernel='linear', random_state=1, C=1.0)
svm_linear.fit(X, y)

# Train non-linear SVC with RBF kernel
svm_nonLinear = SVC(kernel='rbf', random_state=1, gamma=0.10, C=10.0)
svm_nonLinear.fit(X_xor, y_xor)





## Evaluate model

In [ ]:
lr.score(X_test, y_test).compute()

In [ ]:
# Using GridSearchCV to tune hyperparameters
parameters = {'penalty': ['l1', 'l2'], 'C': [0.5, 1, 2]}

lr = LogisticRegression()
tuned_lr = GridSearchCV(lr, parameters)

with ProgressBar():
    tuned_lr.fit(X_train, y_train)

In [ ]:
# Visualising the results of GridSearchCV
pd.DataFrame(tuned_lr.cv_results_)

In [ ]:
# Get hyperparameters of best performing model
tuned_lr.best_params_